In [55]:
import requests
from bs4 import BeautifulSoup
from urllib.request import urlretrieve
from pathlib import Path



In [56]:
test_ds_path = Path("/home/sims/Projects/keras_cv_modules/test_dataset/auto_ria")
test_ds = []
for file in test_ds_path.iterdir():
    if file.suffix != ".jpg":
        continue
    test_ds.append(file.stem)




In [ ]:
for i in range(1, 100):
    print(f"Current step: {i}")
    url = f'https://auto.ria.com/uk/search/?indexName=auto&plateNumber.length.gte=1&country.import.usa.not=-1&price.currency=1&abroad.not=0&custom.not=1&page={i}&size=100&scrollToAuto=37116047'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'lxml')
    items = soup.find_all('section', class_='ticket-item')
    for item in items:
        try:
            sub_url = item.find('a', class_='photo-185x120')["href"]
            # print(sub_url)
            sub_response = requests.get(sub_url)
            sub_item = BeautifulSoup(sub_response.text, 'lxml')
            num = sub_item.find("span", class_="state-num ua").text
            num = num.replace("Ми розпізнали держномер авто на фото та перевірили його за реєстрами МВС.", "").replace(" ", "")
        except:
            continue
        if num in test_ds:
            continue
        imgs = sub_item.find_all('img', class_='outline m-auto')
        imgs = set(img["src"].replace("s.jpg", "f.jpg") for img in imgs)

        for i, ssub_url in zip(range(len(imgs)), imgs):
            name = "" if i >= 10 else "0"
            name += f"{i} {num}.jpg"
            # print(name)
            urlretrieve(ssub_url, f"photos3/{name}")
